In [11]:
import mido

# Path to your MIDI file
midi_file_path = '/Users/zhangyuhong/Desktop/cs1470/final_project/EMOPIA_2.2/midis/Q1__8v0MFBZoco_1.mid'

# Load the MIDI file
mid = mido.MidiFile(midi_file_path)

# Iterate over all messages in all tracks
for i, track in enumerate(mid.tracks):
    print(f'Track {i}: {track.name}')
    for msg in track:
        print(msg)

Track 0: 
Track 1: 


In [10]:
import pygame
import time

def play_midi_file(midi_file_path):
    # Initialize pygame mixer
    pygame.mixer.init()
    
    # Initialize pygame
    pygame.init()
    
    # Load the MIDI file
    pygame.mixer.music.load(midi_file_path)
    
    # Play the MIDI file
    pygame.mixer.music.play()
    
    # Since playback is asynchronous, keep the script running until playback finishes
    while pygame.mixer.music.get_busy():
        pygame.time.wait(100)  # Check every 100ms

# Replace 'path_to_your_midi_file.mid' with the path to your MIDI file

play_midi_file(midi_file_path)


## Diff shape of input

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import to_categorical

latent_dim = 64
input_shape = (None, 3)  # Variable length sequences with 3 features (note, velocity, time)
num_classes = 10  # Number of classes for conditioning

# Encoder
inputs = layers.Input(shape=input_shape)
x = layers.LSTM(128)(inputs)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# Decoder
z_cond = layers.Input(shape=(latent_dim + num_classes,))
x = layers.RepeatVector(input_shape[0])(z_cond)
x = layers.LSTM(128, return_sequences=True)(x)
outputs = layers.TimeDistributed(layers.Dense(input_shape[1]))(x)

# CVAE Model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
decoder = Model(z_cond, outputs, name='decoder')

cvae_inputs = [inputs, z_cond]
cvae_outputs = decoder([z, z_cond])
cvae = Model(cvae_inputs, cvae_outputs, name='cvae')

# Loss
kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
cvae.add_loss(kl_loss)

# Compile the model
cvae.compile(optimizer='adam')




TypeError: Expected an integer value for `n`, got <class 'NoneType'>.

In [ ]:
# Example usage
# Generate some dummy data
num_samples = 1000
max_seq_length = 20

sequences = []
for _ in range(num_samples):
    seq_length = np.random.randint(5, max_seq_length + 1)
    sequence = np.random.rand(seq_length, 3)  # Random sequences of variable lengths
    sequences.append(sequence)

sequences = np.array(sequences)

# Generate dummy labels for conditioning
labels = np.random.randint(0, num_classes, size=num_samples)
one_hot_labels = to_categorical(labels, num_classes)

# Train the model
cvae.fit([sequences, np.concatenate([np.zeros((num_samples, latent_dim)), one_hot_labels], axis=1)], sequences, epochs=10, batch_size=32)